In [1]:
import jax.numpy as jnp
import numpy as np
import meshplot as mp
from mesh_processing import precompute_mesh_attributes, compute_mesh_attributes, compute_laplacian_crane, compute_laplacian_sorkine

In [2]:
DATA = './meshes/'
meshfile = 'cow.obj'
#meshfile = 'suzanne.obj'

V, F, NF, E, NV, N, A, VF, D, El, DEl, DE, DA = precompute_mesh_attributes(DATA, meshfile)

d = mp.plot(np.array(V), np.array(F), return_plot=True, shading={'wireframe': True})

/home/max/anaconda3/lib/python3.7/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


Loading in mesh: 4.173292875289917 s
edge connectivity + lens: 0.25501132011413574 s
vertex neighbourhood connectivity: 0.09339570999145508 s
face neighbourhood connectivity: 0.07124567031860352 s
normals + area: 0.09645414352416992 s
dual points: 0.13018441200256348 s
vertex - face adjacency: 0.6323938369750977 s
dual edge computation: 14.96780800819397 s


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.7761263…

In [7]:
L = compute_laplacian_crane(V, E, El, DEl)
# Forward Euler
f0x = V[:, 0]
f0y = V[:, 1]
f0z = V[:, 2]

h = 2.75

fhx = -f0x - h * jnp.matmul(L, f0x)
fhy = -f0y - h * jnp.matmul(L, f0y)
fhz = -f0z - h * jnp.matmul(L, f0z)

fhforward = jnp.vstack([fhx, fhy, fhz]).T * -1

# Backwards Euler
fhx = jnp.linalg.solve(jnp.eye(L.shape[0], dtype=jnp.float32) - h * L, f0x)
fhy = jnp.linalg.solve(jnp.eye(L.shape[0], dtype=jnp.float32) - h * L, f0y)
fhz = jnp.linalg.solve(jnp.eye(L.shape[0], dtype=jnp.float32) - h * L, f0z)

fhbackward = jnp.vstack([fhx, fhy, fhz]).T

d = mp.subplot(np.array(V), np.array(F), c=np.array([1.0, 0.0, 0.0]),  s=[3, 1, 0])
mp.subplot(np.array(fhforward), np.array(F), c=np.array([1.0, 0.0, 0]), s=[3, 1, 1], data=d)
mp.subplot(np.array(fhbackward), np.array(F), c=np.array([1.0, 0.0, 1.0]), s=[3, 1, 2], data=d)